In [35]:
from google.colab import drive
drive.mount('/content/drive')
!cp '/content/drive/MyDrive/dermoscopy_classification.tar.gz' .
!tar -xf dermoscopy_classification.tar.gz
data_dir = '/content/dermoscopy_classification'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class MLProject2Dataset(Dataset):
    def __init__(self, data_dir, metadata_fname='metadata.csv', transform=None):
        self.data_dir = data_dir
        self.transform = transform


        # Load metadata
        metadata_path = os.path.join(data_dir, metadata_fname)
        metadata = pd.read_csv(metadata_path)
        metadata['dx'] = pd.Categorical(metadata['dx']).codes

        num_classes = metadata['dx'].nunique()

        # Create dataframe with image paths and labels
        image_paths = []
        for part_dir in os.listdir(data_dir):
            part_path = os.path.join(data_dir, part_dir)
            if os.path.isdir(part_path):
                for f in os.listdir(part_path):
                    if f.endswith('.jpg'):
                        image_paths.append(os.path.join(part_path, f))

        image_ids = [os.path.splitext(os.path.basename(f))[0] for f in image_paths]

        # Create dataframe with image paths and labels
        image_df = pd.DataFrame({'image_id': image_ids, 'path': image_paths})

        # Merge dataframes on 'image_id'
        self.data_df = pd.merge(image_df, metadata, on='image_id')

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        img_path = self.data_df.iloc[idx]['path']
        label = int(self.data_df.iloc[idx]['dx'])

        # Load image
        img = Image.open(img_path)

        # Apply transformations (if any)
        if self.transform:
            img = self.transform(img)

        # Ensure the image is a tensor
        if not isinstance(img, torch.Tensor):
            img = transforms.ToTensor()(img)

        return img, label


data_dir = '/content/dermoscopy_classification'
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
dataset = MLProject2Dataset(data_dir, transform=transform)

# Example: Print the length of the dataset
print(len(dataset))

# Example: Retrieve and print the first item in the dataset
sample_img, sample_label = dataset[0]
print(sample_img.shape, sample_label)


10015
torch.Size([3, 224, 224]) 2


In [37]:
from torch.utils.data import random_split
from glob import glob

# Assuming you have already defined and instantiated the MLProject2Dataset class
dataset = MLProject2Dataset(data_dir)

# Seed
seed = 42

print(len(dataset))

# Calculate the sizes of each split
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

# Use random_split to create train, validation, and test datasets
train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    lengths=[train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(seed)
)

# Print the sizes of each split
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# create DataLoader instances for each split
from torch.utils.data import DataLoader

batch_size = 32  # adjust this

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


10015
Train dataset size: 6009
Validation dataset size: 1001
Test dataset size: 3005


In [38]:
from torchvision import transforms

# Define the desired size (m, n) for resizing
m, n = 100, 100  # Change these values

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((m, n)),  # Set m and n to desired size
    transforms.ToTensor(),       # Convert image to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image
])

# Apply transformations to the dataset
train_dataset = MLProject2Dataset(data_dir, transform=transform)
val_dataset = MLProject2Dataset(data_dir, transform=transform)
test_dataset = MLProject2Dataset(data_dir, transform=transform)

# Create DataLoader instances for each split
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total
    return accuracy

def train_net(model: nn.Module, trainloader: DataLoader, valloader: DataLoader = None,
              epochs: int = 10, optimizer: optim = None, loss_fn: nn.modules.loss = None,
              device: str = 'cpu', print_period: int = 10) -> None:
    model.to(device)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        total_train_accuracy = 0.0

        for i, data in enumerate(tqdm(trainloader, desc=f"Epoch {epoch + 1}/{epochs}")):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            total_train_accuracy += calculate_accuracy(outputs, labels)

            if (i + 1) % print_period == 0:
                avg_loss = running_loss / print_period
                avg_train_accuracy = total_train_accuracy / print_period

                print(f"[Batch {i + 1}/{len(trainloader)}] Loss: {avg_loss:.4f}, Accuracy: {avg_train_accuracy:.4f}")
                running_loss = 0.0
                total_train_accuracy = 0.0

        if valloader is not None:
            model.eval()
            val_loss = 0.0
            total_val_accuracy = 0.0

            with torch.no_grad():
                for data in tqdm(valloader, desc="Validation"):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = model(inputs)
                    val_loss += loss_fn(outputs, labels).item()
                    total_val_accuracy += calculate_accuracy(outputs, labels)

                avg_val_loss = val_loss / len(valloader)
                avg_val_accuracy = total_val_accuracy / len(valloader)

                print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {avg_val_accuracy:.4f}")

            model.train()

    print("Training completed.")


In [40]:
def test_net(model: nn.Module, testloader: DataLoader, loss_fn: nn.modules.loss = None, device: str = 'cpu'):
    model.to(device)
    model.eval()

    total_loss = 0.0
    total_accuracy = 0.0
    total_samples = 0

    with torch.no_grad():
        for data in tqdm(testloader, desc="Testing"):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, labels) if loss_fn else None

            total_samples += labels.size(0)
            total_loss += loss.item() if loss is not None else 0.0
            total_accuracy += calculate_accuracy(outputs, labels)

    avg_loss = total_loss / len(testloader) if len(testloader) > 0 else None
    avg_accuracy = total_accuracy / total_samples if total_samples > 0 else None

    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {avg_accuracy:.4f}")

    model.train()  # Set the model back to training mode

    return avg_loss, avg_accuracy


In [41]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 12 * 15, 7)  # Adjust the output size

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = self.flatten(x)

        # Dynamically calculate the size for the fully connected layer
        x_size = x.size(1)
        self.fc = nn.Linear(x_size, 7)

        x = self.fc(x)
        return x

# Define the model, loss function, and optimizer
snn_model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(snn_model.parameters(), lr=0.1)

# Train the model
train_net(model=snn_model, trainloader=train_loader, valloader=val_loader,
          epochs=20, optimizer=optimizer, loss_fn=criterion, print_period=100)

# Test the model
test_net(model=snn_model, testloader=test_loader, loss_fn=criterion)


Epoch 1/2:  32%|███▏      | 100/313 [01:19<02:45,  1.28it/s]

[Batch 100/313] Loss: 1.9455, Accuracy: 0.1347


Epoch 1/2:  64%|██████▍   | 200/313 [02:38<01:19,  1.41it/s]

[Batch 200/313] Loss: 1.9461, Accuracy: 0.1216


Epoch 1/2:  96%|█████████▌| 300/313 [03:56<00:09,  1.42it/s]

...

Validation: 100%|██████████| 313/313 [02:40<00:00,  1.95it/s]


Epoch 2/2, Validation Loss: 1.9461, Validation Accuracy: 0.1573
Training completed.


Testing: 100%|██████████| 313/313 [02:47<00:00,  1.87it/s]

Test Loss: 1.9488, Test Accuracy: 0.0043


(1.9488323507979275, 0.004309161258112831)

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.relu5 = nn.ReLU()
        self.bn5 = nn.BatchNorm2d(512)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.global_avg_pooling = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(512, 7)

    def forward(self, x):
        x = self.pool1(self.bn1(self.relu1(self.conv1(x))))
        x = self.pool2(self.bn2(self.relu2(self.conv2(x))))
        x = self.pool3(self.bn3(self.relu3(self.conv3(x))))
        x = self.pool4(self.bn4(self.relu4(self.conv4(x))))
        x = self.pool5(self.bn5(self.relu5(self.conv5(x))))
        x = self.global_avg_pooling(x)
        x = self.flatten(x)

        # Dynamically calculate the size for the fully connected layer
        x_size = x.size(1)
        self.fc = nn.Linear(x_size, 7)

        x = self.fc(x)
        return x

# Define the model, loss function, and optimizer
complex_cnn_model = ComplexCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(complex_cnn_model.parameters(), lr=1e-3)

# Train the model
train_net(model=complex_cnn_model, trainloader=train_loader, valloader=val_loader,
          epochs=2, optimizer=optimizer, loss_fn=criterion, print_period=100)

# Test the model
test_net(model=complex_cnn_model, testloader=test_loader, loss_fn=criterion)


Epoch 1/2:  32%|███▏      | 100/313 [02:25<05:31,  1.56s/it]

[Batch 100/313] Loss: 2.1319, Accuracy: 0.1341


Epoch 1/2:  64%|██████▍   | 200/313 [04:42<02:25,  1.29s/it]

[Batch 200/313] Loss: 2.0971, Accuracy: 0.1516


Epoch 1/2:  96%|█████████▌| 300/313 [07:00<00:18,  1.41s/it]

...

Validation: 100%|██████████| 313/313 [04:10<00:00,  1.25it/s]


Epoch 2/2, Validation Loss: 2.0055, Validation Accuracy: 0.1488
Training completed.


Testing: 100%|██████████| 313/313 [04:02<00:00,  1.29it/s]

Test Loss: 2.0051, Test Accuracy: 0.0045


(2.005093669739013, 0.004515102346480279)

In [43]:
import torchvision.models as models

# Φόρτωση του προεκπαιδευμένου μοντέλου ResNet34
pretrained_resnet = models.resnet34(pretrained=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
from torchvision import transforms

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [45]:
from torch.utils.data import DataLoader, random_split

# Δημιουργία του dataset
dataset = MLProject2Dataset(data_dir)

# Διαχωρισμός του dataset σε train, validation και test
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

# Δημιουργία των dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [46]:
import torch.nn as nn

class TransferLearningModel(nn.Module):
    def __init__(self):
        super(TransferLearningModel, self).__init__()
        # Φορτώνουμε τα βάρη του προεκπαιδευμένου ResNet34
        self.resnet = models.resnet34(pretrained=True)
        # Επιλέγουμε τον τύπο του επιπλέον classifier που θέλουμε
        # (π.χ., Fully Connected Layer με 7 εξόδους για 7 κλάσεις)
        self.fc = nn.Linear(512, 7)

    def forward(self, x):
        x = self.resnet(x)

        # Dynamically calculate the size for the fully connected layer
        x_size = x.size(1)
        self.fc = nn.Linear(x_size, 7)

        x = self.fc(x)
        return x

transfer_learning_model = TransferLearningModel()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(transfer_learning_model.parameters(), lr=0.001, momentum=0.9)

# Εκπαίδευση για χ εποχές
train_net(model=transfer_learning_model, trainloader=train_loader, valloader=val_loader,
          epochs=2, optimizer=optimizer, loss_fn=criterion, print_period=100)

# Αξιολόγηση στο σύνολο δοκιμής
test_net(model=transfer_learning_model, testloader=test_loader, loss_fn=criterion)


Epoch 1/2:   7%|▋         | 16/220 [22:42<4:54:22, 86.58s/it]

In [48]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data_dir, metadata_fname='metadata.csv', transform=None):
        self.data_dir = data_dir
        self.transform = transform


        # Load metadata
        metadata_path = os.path.join(data_dir, metadata_fname)
        metadata = pd.read_csv(metadata_path)
        metadata['dx'] = pd.Categorical(metadata['dx']).codes

        num_classes = metadata['dx'].nunique()

        # Create dataframe with image paths and labels
        image_paths = []
        for part_dir in os.listdir(data_dir):
            part_path = os.path.join(data_dir, part_dir)
            if os.path.isdir(part_path):
                for f in os.listdir(part_path):
                    if f.endswith('.jpg'):
                        image_paths.append(os.path.join(part_path, f))

        image_ids = [os.path.splitext(os.path.basename(f))[0] for f in image_paths]

        # Create dataframe with image paths and labels
        image_df = pd.DataFrame({'image_id': image_ids, 'path': image_paths})

        # Merge dataframes on 'image_id'
        self.data_df = pd.merge(image_df, metadata, on='image_id')

    def __len__(self):
        return len(self.data_df)
    def __getitem__(self, idx):
        img_path = self.data_df.iloc[idx]['path']
        label = int(self.data_df.iloc[idx]['dx'])

        # Προσθήκη δημογραφικών χαρακτηριστικών
        patient_info = self.data_df.iloc[idx][['age', 'gender', 'body_part']]
        age = patient_info['age'] / 100.0  # Κανονικοποίηση της ηλικίας
        gender = pd.get_dummies(patient_info['gender'], prefix='gender')
        body_part = pd.get_dummies(patient_info['body_part'], prefix='body_part')

        demographic_features = torch.cat([torch.tensor(age), torch.tensor(gender.values), torch.tensor(body_part.values)], dim=0)


        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)

        if not isinstance(img, torch.Tensor):
            img = transforms.ToTensor()(img)

        return img, demographic_features, label


In [49]:
import torch.nn as nn

class DemographicModel(nn.Module):
    def __init__(self):
        super(DemographicModel, self).__init__()
        self.linear = nn.Linear(3, 128)  # 3 demographic features, 128 outputs
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x


In [53]:
class CombinedModel(nn.Module):
    def __init__(self):
        super(CombinedModel, self).__init__()
        self.simple_nn = SimpleNN()
        self.demographic_model = DemographicModel()

        # Define the final linear layer
        final_output_size = 7
        self.final_linear = nn.Linear(128 + final_output_size, final_output_size)

    def forward(self, image, demographic_features):

        # Forward pass through SimpleNN
        x_image = self.simple_nn(image)

        # Forward pass through DemographicModel
        x_demo = self.demographic_model(demographic_features)

        # Concatenate the outputs
        x_combined = torch.cat((x_image, x_demo), dim=1)

        # Final linear layer
        x_final = self.final_linear(x_combined)

        return x_final

